# Peer-graded Assignment Week 3 Part 3

## 1. Import Libraries

In [1]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2. Scrape data from website

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
r_text = r.text

In [3]:
soup = BeautifulSoup(r_text, 'html.parser')

In [4]:
soup = soup.body.table.tbody

In [5]:
df = pd.DataFrame(columns=range(0,3), index = range(0,len(soup.find_all('tr'))))

row_marker = 0
for row in soup.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

df.head()

,0,1,2
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


## 3. Data Preprocessing

In [6]:
df = df.replace(r'\n',' ', regex=True)
df = df.replace(r'Not assigned',np.NaN,regex=True)

df.rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'},inplace=True)
df.dropna(axis=0,subset=['Borough'],inplace=True)
df.reset_index(drop=True,inplace=True)

df['Neighborhood'] = df['Neighborhood'].replace(np.NaN,df['Borough'])

df.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
df_gr = df.groupby(by=['PostalCode','Borough']).Neighborhood.unique().apply(lambda x: ', '.join(x)).reset_index()
df_gr.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [8]:
df_gr.shape

(103, 3)

## 4. Add Lat/Long Data

In [9]:
LatLong = pd.read_csv('Geospatial_Coordinates.csv')
LatLong.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [10]:
LatLong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_merged = pd.merge(df_gr, LatLong, on='PostalCode')
df_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


## 5. Explore and cluster the neighborhoods in Toronto

### 5.1 Explore Dataset

In [12]:
address = 'Toronto City, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6449033, -79.3818364.


In [13]:
# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, post in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood'], df_merged['PostalCode']):
    label = '{}, {}'.format(post, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = '1IIMSWR54DHDW1TBJCVEUPJRMUOOT1V12EAVAKGQ3VVQTVA0' # your Foursquare ID
CLIENT_SECRET = 'UWVK2RNVMJBJC2GQWGZEKHAGSFRENRU4M5YJGJ35LBRR1UXE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Your credentails:
CLIENT_ID: 1IIMSWR54DHDW1TBJCVEUPJRMUOOT1V12EAVAKGQ3VVQTVA0
CLIENT_SECRET:UWVK2RNVMJBJC2GQWGZEKHAGSFRENRU4M5YJGJ35LBRR1UXE


In [15]:
toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Rouge , Malvern 
Highland Creek , Rouge Hill , Port Union 
Guildwood , Morningside , West Hill 
Woburn 
Cedarbrae 
Scarborough Village 
East Birchmount Park , Ionview , Kennedy Park 
Clairlea , Golden Mile , Oakridge 
Cliffcrest , Cliffside , Scarborough Village West 
Birch Cliff , Cliffside West 
Dorset Park , Scarborough Town Centre , Wexford Heights 
Maryvale , Wexford 
Agincourt 
Clarks Corners , Sullivan , Tam O'Shanter 
Agincourt North , L'Amoreaux East , Milliken , Steeles East 
L'Amoreaux West 
Upper Rouge 
Hillcrest Village 
Fairview , Henry Farm , Oriole 
Bayview Village 
Silver Hills , York Mills 
Newtonbrook , Willowdale 
Willowdale South 
York Mills West 
Willowdale West 
Parkwoods 
Don Mills North 
Flemingdon Park , Don Mills South 
Bathurst Manor , Downsview North , Wilson Heights 
Northwood Park , York University 
CFB Toronto , Downsview East 
Downsview West 
Downsview Central 
Downsview Northwest 
Victoria Village 
Woodbine Gardens , Parkview Hill 
Woodbine Heights 
Th

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(2238, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge , Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide , King , Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North , L'Amoreaux East , Milliken , Steeles East",3,3,3,3,3,3
"Albion Gardens , Beaumond Heights , Humbergate , Jamestown , Mount Olive , Silverstone , South Steeles , Thistletown",9,9,9,9,9,9
"Alderwood , Long Branch",8,8,8,8,8,8
"Bathurst Manor , Downsview North , Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57


In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


### 5.2 Analyze Each Neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [20]:
toronto_grouped.shape

(100, 278)

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Coffee Shop,Café,American Restaurant,Bar,Thai Restaurant,Steakhouse,Cosmetics Shop,Burger Joint,Bakery,Hotel
1,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Print Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,"Agincourt North , L'Amoreaux East , Milliken ,...",Coffee Shop,Playground,Park,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens , Beaumond Heights , Humbergate...",Grocery Store,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Coffee Shop,College Rec Center,College Stadium
4,"Alderwood , Long Branch",Pizza Place,Pharmacy,Skating Rink,Gym,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,Gift Shop


### 5.3 Clustering Neighborhoods

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,2.0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,2.0,Electronics Store,Spa,Pizza Place,Rental Car Location,Medical Center,Intersection,Breakfast Spot,Mexican Restaurant,Discount Store,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Dog Run,Dim Sum Restaurant,Diner


In [30]:
toronto_merged.dropna(axis=0,inplace=True)

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5.4 Explore Clusters

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,0.0,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store
97,North York,0.0,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Coffee Shop,Playground,Park,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
23,North York,1.0,Park,Convenience Store,Bank,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
25,North York,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
30,North York,1.0,Park,Airport,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
40,East York,1.0,Park,Convenience Store,Coffee Shop,Rental Car Location,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
44,Central Toronto,1.0,Bus Line,Park,Swim School,Lake,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
48,Central Toronto,1.0,Tennis Court,Park,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
50,Downtown Toronto,1.0,Park,Trail,Playground,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
64,Central Toronto,1.0,Trail,Park,Sushi Restaurant,Jewelry Store,Bus Line,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
74,York,1.0,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport,Department Store,Ethiopian Restaurant,Empanada Restaurant,Adult Boutique


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,Scarborough,2.0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store
2,Scarborough,2.0,Electronics Store,Spa,Pizza Place,Rental Car Location,Medical Center,Intersection,Breakfast Spot,Mexican Restaurant,Discount Store,Dessert Shop
3,Scarborough,2.0,Coffee Shop,Korean Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
4,Scarborough,2.0,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Dog Run,Dim Sum Restaurant,Diner
5,Scarborough,2.0,Playground,Jewelry Store,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
6,Scarborough,2.0,Discount Store,Bus Station,Coffee Shop,Hobby Shop,Department Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
7,Scarborough,2.0,Bus Line,Bakery,Metro Station,Intersection,Bus Station,Soccer Field,Fast Food Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
8,Scarborough,2.0,Motel,American Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
9,Scarborough,2.0,General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,North York,3.0,Pizza Place,Empanada Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4.0,Garden,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
